# License Plate Detection - Data Collection and Exploration

### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image

Test Tensorflow...

In [2]:
print ("Tensorflow {}".format(tf.__version__))

import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Tensorflow 1.14.0
Default GPU Device: /device:GPU:0


### Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

os.chdir(os.path.join(os.getcwd(), 'tf_object_detection'))
working_dir = os.getcwd()

sys.path.append(working_dir)
sys.path.append(working_dir + "/slim")

path = working_dir + ';' + working_dir + '/slim' + ';' + working_dir + '/object_detection'
os.environ['PYTHONPATH'] = path
print ('PYTHONPATH : {}'.format(os.environ['PYTHONPATH']))

os.chdir(os.path.join(os.getcwd(), 'object_detection'))
print ('Working directory : {}'.format(os.getcwd()))

PYTHONPATH : D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection;D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection/slim;D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection/object_detection
Working directory : D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection


### Object detection imports
Here are the imports from the object detection module.

In [4]:
import object_detection

from utils import label_map_util
from utils import visualization_utils as vis_util

In [5]:
from object_detection.utils import ops as utils_ops

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

### Model preparation

#### Variables
Define the dataset and the pretrained model to be used.
To train or test images from the simulator set DATASET=sim for real world images (Carla) set DATASET=real
Change MODEL and MODEL_VERSION if you want to use other pretrained models, see the detection model zoo for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [6]:
# choose pretrained model
MODEL = 'ssdlite_mobilenet_v2_coco_2018_05_09'
PIPELINE_CONFING_FILENAME = 'ssd_mobilenet_v2.config'

NUM_CLASSES = 3

In [7]:
# ****************** don't edit ******************

# What model to download.
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PRETRAIND_MODELS_DIR = 'models'

# List of the strings that is used to add correct label for each box.
DETECTION_COFIG_DIR = '../../config/plate_detection'
PATH_TO_LABELS = os.path.join(DETECTION_COFIG_DIR, 'labels_map.pbtxt')

PIPELINE_CONFING_FILEPATH = os.path.join(DETECTION_COFIG_DIR, PIPELINE_CONFING_FILENAME)

OUTPUT_DIR = '../../output/plate_detection'
CHECKPOINTS_DIR = OUTPUT_DIR + '/checkpoints/' + MODEL
FINETUNED_MODEL_DIR = OUTPUT_DIR + '/fine_tuned_models/' + MODEL

# Path to frozen detection graph.
FINETUNED_MODEL = os.path.join(FINETUNED_MODEL_DIR, 'frozen_inference_graph.pb')

print (PATH_TO_LABELS)
print (PIPELINE_CONFING_FILEPATH)
print (CHECKPOINTS_DIR)
print (FINETUNED_MODEL)

../../config/plate_detection\labels_map.pbtxt
../../config/plate_detection\ssd_mobilenet_v2.config
../../output/plate_detection/checkpoints/ssdlite_mobilenet_v2_coco_2018_05_09
../../output/plate_detection/fine_tuned_models/ssdlite_mobilenet_v2_coco_2018_05_09\frozen_inference_graph.pb


### Download Pretrained Model

**NOTE:** This step only needs to be done once if the pretrained model has not been downloaded yet!

In [10]:
MODEL_ZIP = os.path.join(PRETRAIND_MODELS_DIR, MODEL_FILE)
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_ZIP)
tar_file = tarfile.open(MODEL_ZIP)
tar_file.extractall(os.path.join(os.getcwd(), PRETRAIND_MODELS_DIR))
tar_file.close()

### Train Finetuned Model

Starts the training. This step may take several hours, depending on the computing power of your computer.
You can monitor the training process using tensorboard tensorboard --logdir=%CHECKPOINTS_DIR% and stop the training if once the desired accuracy has been achieved.

In [7]:
%run model_main.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --model_dir={CHECKPOINTS_DIR}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0315 10:18:08.731641 13936 deprecation_wrapper.py:119] From C:\Users\Andreas\Anaconda3\envs\tf1.14\lib\site-packages\object_detection-0.1-py3.7.egg\object_detection\utils\config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



W0315 10:18:08.735638 13936 deprecation_wrapper.py:119] From C:\Users\Andreas\Anaconda3\envs\tf1.14\lib\site-packages\object_detection-0.1-py3.7.egg\object_detection\model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.



W0315 10:18:08.736640 13936 model_lib.py:629] Forced number of epochs for all eval validations to be 1.


W0315 10:18:08.737608 13936 deprecation_wrapper.py:119] From C:\Users\Andreas\Anaconda3\envs\tf1.14\lib\site-packages\object_detection-0.1-py3.7.egg\object_detection\utils\config_util.py:488: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



...

I0315 13:15:08.366266 13936 estimator.py:2099] Saving 'checkpoint_path' summary for global step 20000: ../../output/plate_detection/checkpoints/ssd_mobilenet_v2_coco_2018_03_29\model.ckpt-20000


INFO:tensorflow:Performing the final export in the end of training.


I0315 13:15:08.416249 13936 exporter.py:410] Performing the final export in the end of training.


NotFoundError: Failed to create a directory: ../../output/plate_detection/checkpoints/ssd_mobilenet_v2_coco_2018_03_29\export\Servo\temp-b'1584274508'; No such file or directory

AttributeError: 'function' object has no attribute 'called'

### Export Finetuned Model
If the training is finished, the model must be exported so that it can be used.
Set CHECKPOINT_NO = 'XXX' where XXX is the number of the last checkpoint file in CHECKPOINTS_DIR, before you start the export.

In [7]:
CHECKPOINT_NO = 20000

!python export_inference_graph.py --pipeline_config_path={PIPELINE_CONFING_FILEPATH} --trained_checkpoint_prefix={CHECKPOINTS_DIR}/model.ckpt-{CHECKPOINT_NO} --output_directory={FINETUNED_MODEL_DIR}

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     params
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
param: Number of parameters (in the Variable).

Profile:
node



W0315 14:14:49.108309  6476 deprecation_wrapper.py:119] From export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0315 14:14:49.113308  6476 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0315 14:14:49.113308  6476 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\exporter.py:121: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0315 14:14:49.138308  6476 deprecation_wrapper.py:119] From D:\development\mlnd\German_License_Plate_Recognition\tf_object_detection\object_detection\core\preprocessor.py:2937: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0315 14:14:49.161308  6476 deprecation_wrapper.py:119] F


      FeatureExtractor/MobilenetV2/expanded_conv_10 (--/64.90k params)
        FeatureExtractor/MobilenetV2/expanded_conv_10/depthwise (--/3.46k params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/depthwise/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/depthwise/depthwise_weights (3x3x384x1, 3.46k/3.46k params)
        FeatureExtractor/MobilenetV2/expanded_conv_10/expand (--/24.58k params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/expand/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/expand/weights (1x1x64x384, 24.58k/24.58k params)
        FeatureExtractor/MobilenetV2/expanded_conv_10/project (--/36.86k params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/project/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_10/project/weights (1x1x384x96, 36.86k/36.86k params)
      FeatureExtractor/MobilenetV2/expanded_conv_11 (--/115.78k params)
        FeatureExtr

W0315 14:14:50.175307  6476 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x0000027F313DA088>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x0000027F313DA088>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0315 14:14:50.199307  6476 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x0000027F31456388>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormaliz


          FeatureExtractor/MobilenetV2/expanded_conv_14/depthwise/depthwise_weights (3x3x960x1, 8.64k/8.64k params)
        FeatureExtractor/MobilenetV2/expanded_conv_14/expand (--/153.60k params)
          FeatureExtractor/MobilenetV2/expanded_conv_14/expand/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_14/expand/weights (1x1x160x960, 153.60k/153.60k params)
        FeatureExtractor/MobilenetV2/expanded_conv_14/project (--/153.60k params)
          FeatureExtractor/MobilenetV2/expanded_conv_14/project/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_14/project/weights (1x1x960x160, 153.60k/153.60k params)
      FeatureExtractor/MobilenetV2/expanded_conv_15 (--/315.84k params)
        FeatureExtractor/MobilenetV2/expanded_conv_15/depthwise (--/8.64k params)
          FeatureExtractor/MobilenetV2/expanded_conv_15/depthwise/BatchNorm (--/0 params)
          FeatureExtractor/MobilenetV2/expanded_conv_15/depthwise/depthwise_weight


W0315 14:14:50.790307  6476 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x0000027F31899188>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x0000027F31899188>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0315 14:14:50.837307  6476 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x0000027F3171C248>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting 

### Test Traffic Light Detection

Uses the fine tuned traffic light detection model FINETUNED_MODEL_DIR to detect traffic lights in images.

#### Load the (frozen) finetuned model into memory.

In [8]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(FINETUNED_MODEL, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Loading label map
Loads the label map file PATH_TO_LABELS. This file contains a map that assigns category names to the class-indices:

item { id: 1 name: 'Green' }

item { id: 2 name: 'Yellow' }

item { id: 3 name: 'Red' }

In [9]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [10]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [11]:
import glob

# number of samples to test; -1 <=> all
NUM_SAMPLES = 2

PATH_TO_TEST_IMAGES_DIR = '../../data/plate_detection/test_images'

TEST_IMAGE_PATHS = []
for filename in glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg')):
    TEST_IMAGE_PATHS.append(filename)
    
print ('{} test images found in {}'.format(len(TEST_IMAGE_PATHS), PATH_TO_TEST_IMAGES_DIR))
       
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 12)

5 test images found in ../../data/plate_detection/test_images


In [12]:
if NUM_SAMPLES > 0:
    test_images = np.random.choice(TEST_IMAGE_PATHS, size = NUM_SAMPLES, replace=False)
else:
    test_images = TEST_IMAGE_PATHS

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in test_images:
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections], feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4)
            plt.figure(figsize=IMAGE_SIZE)
            plt.title(image_path)
            plt.imshow(image_np)
            plt.show()

C:\Users\Andreas\Anaconda3\envs\tf1.14\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
